<a href="https://colab.research.google.com/github/asksawant/big-data-derby-2022/blob/main/notebooks/Script1_classifying_race_based_on_number_of_turns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# installing the libraries
!pip install utm
!pip install shapely

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# importing libraries
import pandas as pd
import numpy as np
from datetime import datetime as dt
from tqdm.auto import tqdm
from numpy import nan
import utm
import math

import math
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

import dask.dataframe as dd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:.9f}'.format

In [ ]:
def tqdm_pandas(t):
    from pandas.core.frame import DataFrame
    def inner(df, func, *args, **kwargs):
        t.total = groups.size // len(groups)
        def wrapper(*args, **kwargs):
            t.update(1)
            return func(*args, **kwargs)
        result = df.transform(wrapper, *args, **kwargs)
        t.close()
        return result
    DataFrame.progress_transform = inner

tqdm.pandas(desc="my bar!")

## Reading the files

In [ ]:
# file paths
start_path = '/content/drive/MyDrive/ml-workspace/kaggle/comp/big-data-derby-2022/data/nyra_start_table.csv'
race_path = '/content/drive/MyDrive/ml-workspace/kaggle/comp/big-data-derby-2022/data/nyra_race_table.csv'
tracking_path = '/content/drive/MyDrive/ml-workspace/kaggle/comp/big-data-derby-2022/data/nyra_tracking_table.csv'
complete_path = '/content/drive/MyDrive/ml-workspace/kaggle/comp/big-data-derby-2022/data/nyra_2019_complete.csv'

In [ ]:
sdf = pd.read_csv(start_path,header=None)
rdf = pd.read_csv(race_path)
tdf = pd.read_csv(tracking_path)
cdf = pd.read_csv(complete_path,low_memory=False,index_col=False,header=None)

## Renaming the columns

In [ ]:
# Renaming the start table columns
start_cols = ['track_id','race_date','race_number','program_number','weight_carried','jockey','odds','final_place']
sdf.rename({i: col for i, col in enumerate(start_cols)}, axis=1, inplace=True)

In [ ]:
# Renaming the complete table columns
complete_cols = ['track_id','race_date','race_number','program_number','trakus_index','latitude','longitude','distance_id','course_type','track_condition','run_up_distance','race_type','purse','post_time','weight_carried','jockey','odds','final_place']
cdf.rename({i: col for i, col in enumerate(complete_cols)}, axis=1, inplace=True)

In [ ]:
print("Shape of nyra_2019_complete:",cdf.shape)
print("Shape of nyra_start_table:",sdf.shape)
print("Shape of nyra_race_table:",rdf.shape)
print("Shape of nyra_tracking_table:",tdf.shape)

Shape of nyra_2019_complete: (5228430, 18)
Shape of nyra_start_table: (14915, 8)
Shape of nyra_race_table: (2000, 10)
Shape of nyra_tracking_table: (5228430, 7)


In [ ]:
sdf.head(3)

,track_id,race_date,race_number,program_number,weight_carried,jockey,odds,final_place
0,AQU,2019-01-01,1,1,123,Dylan Davis,130,2
1,AQU,2019-01-01,1,2,120,Junior Alvarado,295,3
2,AQU,2019-01-01,1,3,118,Jose Lezcano,180,4


In [ ]:
cdf.head(3)

,track_id,race_date,race_number,program_number,trakus_index,latitude,longitude,distance_id,course_type,track_condition,run_up_distance,race_type,purse,post_time,weight_carried,jockey,odds,final_place
0,AQU,2019-01-01,9,6,72,40.672901720,-73.827606597,600,D,GD,48,CLM,25000.000000000,420,120,Andre Shivnarine Worrie,2090,8
1,AQU,2019-01-01,9,6,73,40.672945987,-73.827587267,600,D,GD,48,CLM,25000.000000000,420,120,Andre Shivnarine Worrie,2090,8
2,AQU,2019-01-01,9,6,74,40.672990307,-73.827568025,600,D,GD,48,CLM,25000.000000000,420,120,Andre Shivnarine Worrie,2090,8


In [ ]:
tdf.head(3)

,track_id,race_date,race_number,program_number,trakus_index,latitude,longitude
0,AQU,2019-01-01,9,6,72,40.672901720,-73.827606597
1,AQU,2019-01-01,9,6,73,40.672945987,-73.827587267
2,AQU,2019-01-01,9,6,74,40.672990307,-73.827568025


In [ ]:
rdf.head(3)

,track_id,race_date,race_number,distance_id,course_type,track_condition,run_up_distance,race_type,purse,post_time
0,AQU,2019-01-01,1,650,D,MY,36,AOC,80000,1220
1,AQU,2019-01-01,2,600,D,MY,48,MCL,41000,1250
2,AQU,2019-01-01,3,550,D,MY,54,MCL,35000,121


## Cleaning the data

In [ ]:
sdf['program_number'] = sdf['program_number'].apply(lambda x: x.strip())

In [ ]:
cdf['program_number'] = cdf['program_number'].apply(lambda x: x.strip())

## Feature engineering race table

In [ ]:
rdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   track_id         2000 non-null   object
 1   race_date        2000 non-null   object
 2   race_number      2000 non-null   int64 
 3   distance_id      2000 non-null   int64 
 4   course_type      2000 non-null   object
 5   track_condition  2000 non-null   object
 6   run_up_distance  2000 non-null   int64 
 7   race_type        2000 non-null   object
 8   purse            2000 non-null   int64 
 9   post_time        2000 non-null   int64 
dtypes: int64(5), object(5)
memory usage: 156.4+ KB


###Track race distance (Number Turns)
##### AQU - Aqueduct Race Track
http://www.horseracing-tracks.com/tracks/ny/layoutAqu.html
https://www.drf.com/sites/default/files/US%20Track%20Dimensions%20List%20-%20May%202015.pdf
https://en.wikipedia.org/wiki/Aqueduct_Racetrack#:~:text=Inside%20of%20the%20main%20track,and%20total%20capacity%20of%2040%2C000.

After looking at the charts we can conclude that O - Inner Dirt track , D - Main Dirt Track and T - Inner Turf
- Main Track(Dirt) - 1 1/8 miles - 9f
  - 8f,7f,6f,5f - 1 Turn : Below 800 is 1 turn
  - Above 8f - 2 Turn
- Inner Track(Dirt) - 1 miles - 8f
  - 6f : 1 Turn
  - 8f,8.5f : 2 Turn
- Turf - 7f + 43' Around = 7.065f
  - 8f,8.5f: 2 Turn
  - 12f: 3 Turn
  
##### SAR
https://gettingoutofthegate.com/saratoga-turf-primer/
- D: Main Dirt Track
  - 5.5f,6f,7f - 1 Turn
  - 9f,9.5f,10f - 2 Turns
  - 14f - 3 Turns
- T: Outer Turf
  - 5.5f - 1 turn : Below 600
  - 8.5,9 - 2 Turn : Above 800
- M: Hurdle (between T and I)
  - 16.5f - 4 Turn
  - 19f - 5 Turn
- I: Inner Turf
  - 8f,8.5f,9f - 2 Turn
  - 11f,12f - 3 Turn

##### BEL
- D - Main Dirt Track
  - distance <= 900 - 1 Turn
  - 10f,11f,12f - 2 Turn
- T - Outer Turf
  - 6,7f - 1 Turn
  - 8f,8.5f - 1 Turn complete (0.25 Turn*)
  - 11f,12f - 2 Turn 
  - 16f - 3 Turn
- I - Inner Turf
  - 6f - 1 Turn
  - 8.5f - 2 Turn complete (1 is 0.75)
  - 9f,10f,11f - 2 Turn
- M - Herdle ( between T and I)
  - 18f,20F - 4 tURN

In [ ]:
# assigning the number of turns values for AQU
rdf.loc[((rdf['track_id'] == 'AQU') & (rdf['course_type'] == 'D') & (rdf['distance_id'] <= 800)),'num_turns'] = 1
rdf.loc[((rdf['track_id'] == 'AQU') & (rdf['course_type'] == 'D') & (rdf['distance_id'] > 800)),'num_turns'] = 2
rdf.loc[((rdf['track_id'] == 'AQU') & (rdf['course_type'] == 'O') & (rdf['distance_id'] < 800)),'num_turns'] = 1
rdf.loc[((rdf['track_id'] == 'AQU') & (rdf['course_type'] == 'O') & (rdf['distance_id'] >= 800)),'num_turns'] = 2
rdf.loc[((rdf['track_id'] == 'AQU') & (rdf['course_type'] == 'T') & (rdf['distance_id'] <= 900)),'num_turns'] = 2
rdf.loc[((rdf['track_id'] == 'AQU') & (rdf['course_type'] == 'T') & (rdf['distance_id'] >= 1100)),'num_turns'] = 3
# assigning the number of turns values for SAR
rdf.loc[((rdf['track_id'] == 'SAR') & (rdf['course_type'] == 'D') & (rdf['distance_id'] <= 700)),'num_turns'] = 1
rdf.loc[((rdf['track_id'] == 'SAR') & (rdf['course_type'] == 'D') & (rdf['distance_id'] > 700) & (rdf['distance_id'] <= 1000)),'num_turns'] = 2
rdf.loc[((rdf['track_id'] == 'SAR') & (rdf['course_type'] == 'D') & (rdf['distance_id'] >= 1400)),'num_turns'] = 3
rdf.loc[((rdf['track_id'] == 'SAR') & (rdf['course_type'] == 'T') & (rdf['distance_id'] < 600)),'num_turns'] = 1
rdf.loc[((rdf['track_id'] == 'SAR') & (rdf['course_type'] == 'T') & (rdf['distance_id'] >= 800) & (rdf['distance_id'] <= 950)),'num_turns'] = 2
rdf.loc[((rdf['track_id'] == 'SAR') & (rdf['course_type'] == 'M') & (rdf['distance_id'] > 1600) & (rdf['distance_id'] < 1700)),'num_turns'] = 4
rdf.loc[((rdf['track_id'] == 'SAR') & (rdf['course_type'] == 'M') & (rdf['distance_id'] >= 1900) & (rdf['distance_id'] < 2000)),'num_turns'] = 5
rdf.loc[((rdf['track_id'] == 'SAR') & (rdf['course_type'] == 'I') & (rdf['distance_id'] >= 800) & (rdf['distance_id'] <= 900)),'num_turns'] = 2
rdf.loc[((rdf['track_id'] == 'SAR') & (rdf['course_type'] == 'I') & (rdf['distance_id'] >= 1100) & (rdf['distance_id'] <= 1200)),'num_turns'] = 3
# assigning the number of turns values for BEL
rdf.loc[((rdf['track_id'] == 'BEL') & (rdf['course_type'] == 'D') & (rdf['distance_id'] <= 900)),'num_turns'] = 1
rdf.loc[((rdf['track_id'] == 'BEL') & (rdf['course_type'] == 'D') & (rdf['distance_id'] >= 1000) & (rdf['distance_id'] <= 1200)),'num_turns'] = 2
rdf.loc[((rdf['track_id'] == 'BEL') & (rdf['course_type'] == 'T') & (rdf['distance_id'] <= 850)),'num_turns'] = 1
rdf.loc[((rdf['track_id'] == 'BEL') & (rdf['course_type'] == 'T') & (rdf['distance_id'] >= 1100) & (rdf['distance_id'] <= 1200)),'num_turns'] = 2
rdf.loc[((rdf['track_id'] == 'BEL') & (rdf['course_type'] == 'T') & (rdf['distance_id'] >= 1600) & (rdf['distance_id'] <= 1650)),'num_turns'] = 3
rdf.loc[((rdf['track_id'] == 'BEL') & (rdf['course_type'] == 'I') & (rdf['distance_id'] > 550) & (rdf['distance_id'] < 650)),'num_turns'] = 1
rdf.loc[((rdf['track_id'] == 'BEL') & (rdf['course_type'] == 'I') & (rdf['distance_id'] > 800) & (rdf['distance_id'] <= 1100)),'num_turns'] = 2
rdf.loc[((rdf['track_id'] == 'BEL') & (rdf['course_type'] == 'M') & (rdf['distance_id'] >= 1800) & (rdf['distance_id'] <= 2000)),'num_turns'] = 4

In [ ]:
rdf['num_turns'] = rdf['num_turns'].astype('int16')

In [ ]:
rdf.head(3)

,track_id,race_date,race_number,distance_id,course_type,track_condition,run_up_distance,race_type,purse,post_time,num_turns
0,AQU,2019-01-01,1,650,D,MY,36,AOC,80000,1220,1
1,AQU,2019-01-01,2,600,D,MY,48,MCL,41000,1250,1
2,AQU,2019-01-01,3,550,D,MY,54,MCL,35000,121,1
